# Forces 🏋️

- [ForceGraph Top level API](https://github.com/vasturiano/force-graph#force-engine-d3-force-configuration)
  - Forcing Functions:
    - `d3Force(str, [fn])`
    - Default forces: {"link", "charge", "center"}
    - Example of
      ["collide" and "box"](https://vasturiano.github.io/force-graph/example/collision-detection/)([source](https://github.com/vasturiano/force-graph/blob/master/example/collision-detection/index.html))
    - https://www.d3indepth.com/force-layout/
      - forceCenter
      - forceCollide
        - [using mouse](https://observablehq.com/@d3/collision-detection/2?collection=@d3/d3-force)
      - forceLink
      - forceManyBody
      - forceRadial
      - forceX
      - forceY
    - [full on custom](https://technology.amis.nl/frontend/introduction-to-d3-force-for-simulation-and-animation/)
      - use handlebar templates?
  - [DAG](https://github.com/vasturiano/force-graph/blob/master/example/tree/index.html)

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import asyncio
import random
from typing import Dict

import ipywidgets as W
import traitlets as T

import ipyforcegraph.forces as F

In [ ]:
with __import__("importnb").Notebook():
    import Behaviors as B
    import Utils as U


node_template_help = {"context": "node", "context_all": "nodes"}
link_template_help = {"context": "link", "context_all": "links"}


def make_slider_template(
    force: F.BaseD3Force,
    attr: str,
    template_help: Dict[str, str],
    slider_kwargs: dict = None,
):
    """Make a slider and template combo and link to given `force`'s `attr`."""
    slider_kwargs = slider_kwargs if slider_kwargs is not None else {}
    if "description" not in slider_kwargs:
        slider_kwargs["description"] = attr.title()
    if "layout" not in slider_kwargs:
        slider_kwargs["layout"] = {"flex": "1"}
    placeholder = (
        "{{{{{context}}}}}, {{{{i}}}}, {{{{{context_all}}}}} are defined".format(
            **template_help
        )
    )

    ui_slider = W.FloatSlider(**slider_kwargs)
    ui_template = W.Textarea(layout=dict(max_width="100%"), placeholder=placeholder)
    T.dlink(
        (ui_template, "value"),
        (ui_slider, "disabled"),
        lambda x: True if x else False,
    )
    T.dlink((ui_template, "value"), (force, attr))
    T.dlink((ui_slider, "value"), (force, attr), str)
    return [ui_slider, ui_template]

In [ ]:
if __name__ == "__main__":
    fg, box = U.make_a_demo()
    box.description = "Force Demo"
    display(box)

## `GraphForces`

If the built-in forces do not meet the needs of a particular application, these can be
customized in many ways. These are collected under the `GraphForces` behavior, which has
both top-level parameters, common to the overall simulation engine, as well is
individual named forces inside of it, which are themselves highly configurable.

### Warmup and Cooldown

These `warmup_ticks` and `cooldown_ticks` parameters control how the simulation starts
up and how long it is allowed to run.

By default, `cooldown_ticks` is set to `-1`, meaning the simulation will be allowed to
run as long as it needs before reaching a steady state.

When `cooldown_ticks` is set to `0`, the simulation won't run at all, useful for
pre-calculated positions.

`warmup_ticks` controls how much is simulated off-screen, before any force-driven
animation begins.

In [ ]:
def add_force_ticks(fg, box):
    gf = U.ensure_graph_forces(fg, box)

    warmup = W.IntSlider(description="warmup", min=0, max=100)
    cooldown = W.IntSlider(description="cooldown", min=-1, max=100)

    T.link((gf, "warmup_ticks"), (warmup, "value"))
    T.link((gf, "cooldown_ticks"), (cooldown, "value"))

    ui_ticks = U.make_a_collapsible_picker(
        "graph: warmup & cooldown",
        {
            "spoon": [W.VBox([warmup, cooldown], layout=dict(width="100%"))],
        },
    )
    box.node_ui = {**box.node_ui, "graph_ticks": ui_ticks}
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_force_ticks(fg, box)

### Alpha and Velocity

The parameters `alpha_min`, `alpha_decay` and `velocity_decay` fine-tune the natural
stopping state of the simulation.

In [ ]:
def add_alpha_velocity(fg, box):
    gf = U.ensure_graph_forces(fg, box)

    alpha_min = W.FloatSlider(
        description="alpha min", min=0, max=1, step=0.0001, readout_format=".4f"
    )
    alpha_decay = W.FloatSlider(
        description="alpha decay", min=0, max=1, step=0.0001, readout_format=".4f"
    )
    velocity_decay = W.FloatSlider(
        description="velocity decay", min=0, max=1, step=0.0001, readout_format=".4f"
    )

    T.link((gf, "alpha_min"), (alpha_min, "value"))
    T.link((gf, "alpha_decay"), (alpha_decay, "value"))
    T.link((gf, "velocity_decay"), (velocity_decay, "value"))

    ui_ticks = U.make_a_collapsible_picker(
        "graph: alpha & velocity",
        {
            "av": [
                W.VBox(
                    [alpha_min, alpha_decay, velocity_decay], layout=dict(width="100%")
                )
            ],
        },
    )
    box.node_ui = {**box.node_ui, "graph_alpha_velocity": ui_ticks}
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_alpha_velocity(fg, box)

## Base Forces

By default the forcegraph uses the following forces:

- `link` - LinkForce
- `charge` - ManyBodyForce
- `force_center` - CenterForce

In [ ]:
def add_charge_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force = F.ManyBodyForce()
    gf.forces = {**gf.forces, "charge": force}

    sliders = dict(
        distance_min=W.FloatSlider(
            description="Min Distance", min=0, value=0, max=200, layout={"flex": "1"}
        ),
        distance_max=W.FloatSlider(
            description="Max Distance",
            min=0,
            value=20000,
            max=20000,
            layout={"flex": "1"},
        ),
    )

    for trait, slider in sliders.items():
        T.link((slider, "value"), (force, trait))

    controls = [
        *sliders.values(),
        *make_slider_template(
            force,
            attr="strength",
            template_help=node_template_help,
            slider_kwargs={"min": -60, "value": -30, "max": 2},
        ),
    ]

    force_ui, children = U.ensure_graph_force_ui(
        fg, box, {"charge": [W.VBox(controls, layout={"flex": "1"})]}
    )
    return fg, box


def add_link_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force = F.LinkForce()
    gf.forces = {**gf.forces, "link": force}

    controls = [
        *make_slider_template(
            force,
            attr="distance",
            template_help=link_template_help,
            slider_kwargs={"min": 0, "value": 30, "max": 100},
        ),
        *make_slider_template(
            force,
            attr="strength",
            template_help=link_template_help,
            slider_kwargs={"min": 0, "value": 0.1, "max": 1},
        ),
    ]

    force_ui, children = U.ensure_graph_force_ui(
        fg, box, {"link": [W.VBox(controls, layout={"flex": "1"})]}
    )
    return fg, box


def add_center_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force = F.CenterForce()
    gf.forces = {**gf.forces, "center": force}
    # Center UI
    center_sliders = dict(
        x=W.FloatSlider(description="X", min=-200, max=200, layout={"flex": "1"}),
        y=W.FloatSlider(description="Y", min=-200, max=200, layout={"flex": "1"}),
        z=W.FloatSlider(description="Z", min=-200, max=200, layout={"flex": "1"}),
    )

    for key in ["x", "y", "z"]:
        T.link((center_sliders[key], "value"), (force, key))
    controls = list(center_sliders.values())

    force_ui, children = U.ensure_graph_force_ui(
        fg, box, {"center": [W.VBox(controls, layout={"flex": "1"})]}
    )
    return fg, box


if __name__ == "__main__":
    fg, box = add_charge_force(fg, box)
    fg, box = add_link_force(fg, box)
    fg, box = add_center_force(fg, box)

## `CollisionForce`

The collision force treats nodes as circles with a given `radius`, rather than points
and prevents nodes from overlapping.

In [ ]:
def add_collide_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force = F.CollisionForce(radius="4")
    gf.forces = {**gf.forces, "collide": force}

    strength_slider = W.FloatSlider(
        description="Strength", min=0, value=1, max=1, layout={"flex": "1"}
    )
    controls = [
        strength_slider,
        *make_slider_template(
            force,
            attr="radius",
            template_help=node_template_help,
            slider_kwargs={"min": 0, "value": int(force.radius), "max": 100},
        ),
    ]

    T.link((strength_slider, "value"), (force, "strength"))
    children = {
        "collide": [W.VBox(controls, layout={"flex": "1"})],
    }
    force_ui, children = U.ensure_graph_force_ui(fg, box, children)

    return fg, box

In [ ]:
if __name__ == "__main__":
    add_collide_force(fg, box)

## `RadialForce`

The radial positioning force create a force towards a circle of the specified radius
centered at (x, y).

In [ ]:
def add_radial_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force = F.RadialForce(radius="100", strength="0")

    gf.forces = {**gf.forces, "radial": force}
    center_sliders = dict(
        x=W.FloatSlider(description="X", min=-200, max=200, layout={"flex": "1"}),
        y=W.FloatSlider(description="Y", min=-200, max=200, layout={"flex": "1"}),
        z=W.FloatSlider(description="Z", min=-200, max=200, layout={"flex": "1"}),
    )

    for key, slider in center_sliders.items():
        T.link((slider, "value"), (force, key))

    controls = [
        *make_slider_template(
            force,
            attr="strength",
            template_help=node_template_help,
            slider_kwargs={"min": -10, "value": 0, "max": 10},
        ),
        *make_slider_template(
            force,
            attr="radius",
            template_help=node_template_help,
            slider_kwargs={"min": 0, "value": 10, "max": 200},
        ),
    ]

    children = {
        "radial": [
            W.VBox(
                [
                    *center_sliders.values(),
                    *controls,
                ],
                layout={"flex": "1"},
            )
        ],
    }
    force_ui, children = U.ensure_graph_force_ui(fg, box, children)

    return fg, box


if __name__ == "__main__":
    fg, box = add_radial_force(fg, box)

## Cluster Force

A force type that attracts nodes toward a set of cluster centers.

`centers` - Defines each node's cluster center. All cluster centers should be defined as
a radius and set of coordinates { radius, x, y, z }, according to the number of spatial
dimensions in the simulation.

In [ ]:
def add_cluster_force(fg, box):
    gf = U.ensure_graph_forces(fg, box)
    force = F.ClusterForce()

    gf.forces = {**gf.forces, "cluster": force}
    center_inertia = W.FloatSlider(
        description="Inertia", min=0, value=0, max=1, layout={"flex": "1"}
    )
    strength = W.FloatSlider(
        description="Strength", min=0, value=0, max=1, layout={"flex": "1"}
    )
    centers_template = W.Textarea(
        layout=dict(flex="1", max_width="100%"), placeholder="cluster center template"
    )
    T.link((center_inertia, "value"), (force, "center_inertia"))
    T.link((strength, "value"), (force, "strength"))
    T.link((centers_template, "value"), (force, "centers"))

    controls = [
        center_inertia,
        strength,
        centers_template,
    ]

    children = {
        "cluster": [
            W.VBox(
                controls,
                layout={"flex": "1"},
            )
        ],
    }
    force_ui, children = U.ensure_graph_force_ui(fg, box, children)

    return fg, box


if __name__ == "__main__":
    fg, box = add_cluster_force(fg, box)

Positioning Forces X, Y, and Z will create a force pushing/pulling nodes along given
axis position.

In [ ]:
def config_pos_force(axis: str):
    force_map = {
        "x": F.XForce,
        "y": F.YForce,
        "z": F.ZForce,
    }

    force = force_map[axis]()

    def add_pos_force(fg, box):
        gf = U.ensure_graph_forces(fg, box)

        gf.forces = {**gf.forces, axis: force}
        controls = [
            *make_slider_template(force, attr=axis, template_help=node_template_help),
            *make_slider_template(
                force, attr="strength", template_help=node_template_help
            ),
        ]

        children = {
            axis: [
                W.VBox(
                    controls,
                    layout={"flex": "1"},
                )
            ],
        }
        force_ui, children = U.ensure_graph_force_ui(fg, box, children)
        return fg, box

    return add_pos_force

In [ ]:
add_x_force = config_pos_force("x")
add_y_force = config_pos_force("y")
add_z_force = config_pos_force("z")
if __name__ == "__main__":
    fg, box = add_x_force(fg, box)
    fg, box = add_y_force(fg, box)
    fg, box = add_z_force(fg, box)

In [ ]:
add_base_forces = [
    add_link_force,
    add_charge_force,
    add_center_force,
]
all_forces = [
    add_force_ticks,
    add_alpha_velocity,
    *add_base_forces,
    add_collide_force,
    add_radial_force,
    add_cluster_force,
    add_x_force,
    add_y_force,
    add_z_force,
]

# Tests

These tests are executed in the [Force Tests Demo](./Test_Forces.ipynb) notebook.

In [ ]:
import pandas as pd


async def get_positions(fg, box) -> pd.DataFrame:
    graph_data = box.behaviors["graph_data"]
    graph_data.capturing = True
    data = await U.wait_for_change(graph_data.sources[0], "nodes")
    return data


def check_center(pos: pd.DataFrame, x=0, y=0, tolerance=0.1):
    x0, y0 = map(lambda s: s.mean(), [pos.x, pos.y])
    delta_x = tolerance * (pos.x.max() - pos.x.min())
    delta_y = tolerance * (pos.y.max() - pos.y.min())

    errors = []

    if abs(x0 - x) > delta_x:
        errors.append(
            f"Expecting x position to be near {x}. \n\tNot x={x0} \n\tdelta_x={delta_x}"
        )
    if abs(y0 - y) > delta_y:
        errors.append(
            f"Expecting y position to be near {y}. \n\tNot y={y0} \n\tdelta_y={delta_y}"
        )
    assert len(errors) == 0, "\n".join(errors)

## Center Force Test

In [ ]:
async def test_center_force(fg, box):
    """Varies the x and y coordinate for the center force"""
    gf = U.ensure_graph_forces(fg, box)
    force = gf.forces["center"]

    force.x = 100
    force.y = 100
    box.description = f"Moving center to ({force.x}, {force.y})"
    await asyncio.sleep(4)
    pos = await get_positions(fg, box)

    check_center(
        pos,
        force.x,
        force.y,
    )

    force.x = 0
    force.y = 0
    box.description = f"Moving center to ({force.x}, {force.y})"
    await asyncio.sleep(4)
    pos = await get_positions(fg, box)
    check_center(
        pos,
        force.x,
        force.y,
    )

## Positioning Force Tests

In [ ]:
def test_pos_force(axis: str):
    assert axis in ["x", "y"], f"Invalid pos axis `{axis}`"
    off_axis = "y" if axis == "x" else "x"

    async def test_force(fg, box):
        gf = U.ensure_graph_forces(fg, box)
        force = gf.forces[axis]
        force.strength = "1"
        axis_pos = 10
        setattr(force, axis, str(axis_pos))
        box.description = f"Moving axis to {axis}={axis_pos}"
        await asyncio.sleep(4)
        pos = await get_positions(fg, box)
        center = pos[axis].mean()
        assert (center - axis_pos) / (
            pos[axis].max() - pos[axis].min()
        ) < 0.01, f"Expecting center near {axis_pos} not {center}"

        axis_pos = 200
        setattr(force, axis, str(axis_pos))
        box.description = f"Moving axis to {axis}={axis_pos}"
        await asyncio.sleep(4)
        pos = await get_positions(fg, box)
        center = pos[axis].mean()
        assert (center - axis_pos) / (
            pos[axis].max() - pos[axis].min()
        ) < 0.01, f"Expecting center near {axis_pos} not {center}"

        # test the spread of y goes down after reseting the force strength
        std_off_axis = pos[off_axis].std()
        force.strength = None
        box.description = f"Testing spread on {off_axis}."
        await asyncio.sleep(4)
        pos = await get_positions(fg, box)

        assert (
            std_off_axis > pos[off_axis].std()
        ), f"Expecting `{off_axis}` spread to be smaller after resetting the force"

    test_force.__doc__ = f"Changing the position of the Force on the {axis} axis."
    return test_force

In [ ]:
all_force_tests = [test_center_force, test_pos_force("x"), test_pos_force("y")]